# KNN demo
K-Nearest Neighbor is a supervised ML algorithm with a target variable (y) depending on independent variables (X) used for prediction.
In this algorithm, the entire training dataset is stored. When a prediction is required, the k-most similar records to a new record from the training dataset are then located. From these neighbors, a summarized prediction is made.

**accompanying slides:**
https://docs.google.com/presentation/d/1jNR_1fJYOuR8qoLcwh7M07zZB2ObDOufiHQtOVJKuNU/edit?usp=sharing

### Import libraries and initialize random generator

In [ ]:
import pandas as pd
import numpy as np
# Set the seed value to make the random number reproducible
np.random.seed(0)

### Read data from CSV
we will use the flower iris dataset. This data sets consists of 3 different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length

In [ ]:
#read in dataset
dataset = pd.read_csv("/kaggle/input/iris/Iris.csv")
display(dataset)

here we split the data into input variable (y) and output variables (X).

In [ ]:
#Split the dataset into X (input features) and y (output labels)
X = dataset[ ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'] ].values
y = dataset['Species'].values

In [ ]:
# first 5 rows of X:
display(X[0:5])

In [ ]:
# first 5 rows of y:
display(y[0:5])

The labels are in strings, so we will convert them into integers
By converting strings to numbers, the models can process the data more efficiently, and perform comparisons on the labels

In [ ]:
#converting text labels to integers
print("Before:",y)
unique_label_text, y_labels_integer = np.unique(y, return_inverse=True)
print("After:",y_labels_integer)

### Split the dataset into training sets and testing sets
this is usually done using `scipy` library's `train_test_split` function:
`X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)`

However, this tutorial will explicitly show the processes of splitting datasets.

This code segment splits a dataset into training and testing sets for machine learning purposes.
This splitting process is commonly used in machine learning to assess model performance on unseen data, allowing for effective training and evaluation.

- An array of indices, `indices`, which is integers from 0 to 149, is created.
- The `np.random.shuffle(indices)` line shuffles the indices to randomize the data order. If we do not randomize and just take the first thirty samples as testing set, they will all be the same type of iris.

In [ ]:
indices = np.arange(len(X))  # Create an array of indices for shuffling
print(indices)

In [ ]:
np.random.shuffle(indices)  # Shuffle the indices to randomize the data
print(indices)

- The `split_ratio` variable determines the ratio of the training set to the entire dataset.
- The `index_to_split` variable calculates the index at which the dataset will be split based on the ratio.

In [ ]:
split_ratio = 0.8  # Ratio of training set vs. whole dataset
index_to_split = int(len(X) * split_ratio)  # Determine the index to split the dataset
print(index_to_split)

- The subsequent commented code block demonstrates an explicit way of splitting the dataset using slicing.
- The lists `X_train`, `X_test`, `y_train`, and `y_test` are initialized to store the training and testing data and labels.
- A loop fill the `X_train` and `y_train` lists with the corresponding data and labels for the training set.

In [ ]:
#initialize the lists which the dataset will be split into
X_train = []
X_test = []
y_train = []
y_test = []

# Populate the training data and labels
for i in range(index_to_split):
    X_train.append( X[indices[i]] )
    y_train.append( y_labels_integer[indices[i]] )

# Populate the testing data and labels
for i in range(index_to_split, len(indices)):
    X_test.append( X[indices[i]] )
    y_test.append( y_labels_integer[indices[i]] )

In [ ]:
X_train

- Finally, the lists are converted to 2D numpy arrays for compatibility with machine learning algorithms.

In [ ]:
# Convert the lists to 2d numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
print("X_test",X_test)
print("y_test",y_test)

### KNN classifier function
This k-Nearest Neighbors tutorial is broken down into 3 parts:

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*CVcFTGsIZ-sj_Z_CY5X0Ew.png" width=50%>

#### Step 1: Calculate Euclidean Distance.
- We can calculate the straight line distance between two vectors using the Euclidean distance measure. It is calculated as the square root of the sum of the squared differences between the two vectors.
- To locate the neighbors for a new piece of data within a dataset we must first calculate the distance between each record in the dataset to the new piece of data (note in the image not all arrows are shown)

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*0ObiRFulLMuXQ5aj6dO7BQ.png" width=50%>

#### Step 2: Get Nearest Neighbors.
- Neighbors for a new piece of data in the dataset are the k closest instances, as defined by our distance measure.
- Once distances are calculated, we must sort all of the records in the training dataset by their distance to the new data. We can then select the top k to return as the most similar neighbors.

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*El2DdaYNYaBficIL7EXwxg.png" width=50%>

#### Step 3: Make Predictions.
- The most similar neighbors collected from the training dataset is used to make predictions.
- `np.unique` is used to find the corresponding counts of each unique label type
- `np.argmax` is used to find the indices of the maximum values

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*TyqUebB-B9MN-XhcKmVGWA.png" width=50%>


In [ ]:
def KNNClassify(X_test, Y_train = y_train,X_train = X_train, k = 8): #set default k as 8
    # X_test is the data of 1 flower
    # X_train is the data of 120 flowers

    ## Step 1:
    distance_list = []

    #for every example in the training set, calculate euclidean distance against the test example
    for i in range(len(X_train)):
        point = X_train[i]
        d1 = (point[0]-X_test[0])**2 # 5**2 means 5^2
        d2 = (point[1]-X_test[1])**2
        d3 = (point[2]-X_test[2])**2
        d4 = (point[3]-X_test[3])**2
        distance = np.sqrt(d1+d2+d3+d4)

        #place this calculated distance into the list
        distance_list.append( (distance,i) )

    ## Step 2:
    #sort distances in ascending order by distance
    sorted_distance_list = sorted(distance_list)

    #the k nearest neighbours are the top k points in the sorted distance list
    neighbours = sorted_distance_list[0:k]
    # for example, neighbours = [(352.1, 51),(371.2, 9),(373.5, 2),(376.0, 110),]

    #get index of the minimum distances
    neighbours_index = []
    for distance,idx in neighbours:
        neighbours_index.append(idx)
    # for example, neighbours_index = [51, 9, 2, 110]

    ## Step 3:
    #check which label has majority
    output = Y_train[neighbours_index]
    # for example, neighbours_index = [1, 0, 0, 2] because #51 is iris_type1, #9 and #2 is iris_type0, #110 is iris_type2
    values, counts = np.unique(output, return_counts=True)

    #return label with majority occurence
    max_idx = np.argmax(counts)
    return values[max_idx]

### Running the classifier to predict the labels for the testing set

In [ ]:
#getting predicted values using our algorithm
predicted_y = []
for point in X_test:
    predicted_y.append(KNNClassify(point))

print(predicted_y)

In [ ]:
def accuracy(predictions , y_test):
    count = 0
    for i in range(len(predictions)):
        if predictions[i] == y_test[i]:
            count +=1
    return count/len(predictions)


#compare the predictions against the y_tests (true labels）
acc = accuracy(predicted_y, y_test)
print("Accuracy =", acc*100, "%")

### Exercise
1. Alice wants to use KNN to classify cat and dog images, which have 50,000 training images. She wants to distribute this model on the internet. Is this feasible? Why or why not? (hint: think about the size of the trained model)  
2. Explain why there are 4 variables added together in `distance = np.sqrt(d1+d2+d3+d4)`
3. Write the code below so that the KNN classifier can handle 28x28 pixels hand-written greyscale digit image classification. The inputs are vectors of 784 numbers, and the ouput is the label (10 categories)

In [ ]:
def KNNClassify(X_test, Y_train = y_train,X_train = X_train, k = 8):
    distance_list = []
    for idx,point in enumerate(X_train):
        #exercise: code here

        distance_list.append( (distance,idx) )

    sorted_distance_list = sorted(distance_list)

    neighbours = sorted_distance_list[:k]
    neighbours_index = []
    for distance,idx in neighbours:
        neighbours_index.append(idx)

    output = Y_train[neighbours_index]
    values, counts = np.unique(output, return_counts=True)

    max_idx = np.argmax(counts)
    return values[max_idx]

Reference
1. https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/
2. https://medium.com/analytics-vidhya/k-nn-from-scratch-212dcff13eb3
3. https://towardsdatascience.com/create-your-own-k-nearest-neighbors-algorithm-in-python-eb7093fc6339